### Import of necessary libraries

In [1]:
from interface import Interface
from shapely import Polygon
import folium
import geopandas
import json

### Creation of a polygon as an example of AOI

In [2]:
# simplified example of a call to the interface

lat_point_list = [50.854457, 50.924457, 52.518172, 50.072651, 48.853033]
lon_point_list = [4.377184, 4.377184, 13.407759, 14.435935, 2.349553]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))

polygon_geom = geopandas.GeoSeries(polygon_geom)

m = folium.Map(location=[50.854457, 4.377184], zoom_start=5)
folium.GeoJson(polygon_geom.__geo_interface__).add_to(m)

x = geopandas.GeoSeries(polygon_geom)
              
rectangle = geopandas.GeoDataFrame(geometry=x.envelope)

# Define the number of rows and columns for the grid
num_rows = 5
num_cols = 5

config_file = open('config.json')
config_parser = json.load(config_file)
num_cols = config_parser["granularityParameters"]["granularity"]
num_rows = config_parser["granularityParameters"]["granularity"]

# Calculate the width and height of each sub-area
width = (rectangle.bounds['maxx'] - rectangle.bounds['minx']) / num_cols
height = (rectangle.bounds['maxy'] - rectangle.bounds['miny']) / num_rows

# Create an empty list to store the sub-areas
sub_areas = []

# Iterate over the rows and columns to create the grid of sub-areas
for row in range(num_rows):
    for col in range(num_cols):
        minx = rectangle.bounds['minx'] + col * width
        maxx = minx + width
        miny = rectangle.bounds['miny'] + row * height
        maxy = miny + height
        sub_area_coords = [(minx, miny), (maxx, miny), (maxx, maxy), (minx, maxy)]
        sub_areas.append(Polygon(sub_area_coords))

# Create a GeoDataFrame with the sub-areas
sub_areas_gdf = geopandas.GeoDataFrame(geometry=sub_areas)

# Perform the spatial intersection to obtain the divided sub-areas
divided_areas = geopandas.overlay(sub_areas_gdf, rectangle, how='intersection')

# print(divided_areas)

for index, row in divided_areas.iterrows():
    geojson = row.geometry.__geo_interface__
    folium.GeoJson(geojson).add_to(m)
    
centroids = set()

for index, row in divided_areas.iterrows():    
    if(polygon_geom.intersects(row.geometry).bool()==True):
        centroids.add(row.geometry.centroid)

for centroid in centroids:
    # print(centroid)
    folium.GeoJson(centroid).add_to(m)

# folium.GeoJson(divided_areas.to_json).add_to(m)
   
m

### The class interface is created and the method get_weather_forecast is called by group 6

In [3]:
interface = Interface()
interface.get_weather_forecast(polygon_geom)

{'u3', 'u0', 'u1', 'u2'}
<class 'str'>temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_high,visibility,evapotranspiration,et0_fao_evapotranspiration,vapor_pressure_deficit,windspeed_10m,windspeed_80m,windspeed_120m,windspeed_180m,winddirection_10m,winddirection_80m,winddirection_120m,winddirection_180m,windgusts_10m,temperature_80m,temperature_120m,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_1cm,soil_moisture_1_3cm,soil_moisture_3_9cm,soil_moisture_9_27cm,soil_moisture_27_81cm,is_day
<class 'str'>temperature_2m,relativehumidity_2m,dewpoint_2m,apparent_temperature,precipitation_probability,precipitation,rain,showers,snowfall,snow_depth,weathercode,pressure_msl,surface_pressure,cloudcover,cloudcover_low,cloudcover_mid,cloudcover_hi

KeyError: 'hourly'